# Test In Simulation

In [ ]:
from Go2Py.robot.fsm import FSM
from Go2Py.robot.remote import KeyboardRemote
from Go2Py.robot.safety import SafetyHypervisor
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.control.walk_these_ways import *

In [2]:
robot = Go2Sim()

In [3]:
map = np.zeros((1200, 1200))
map[:200, :200] = 255
robot.updateHeightMap(map)

In [4]:
remote = KeyboardRemote()
robot.sitDownReset()
safety_hypervisor = SafetyHypervisor(robot)

In [5]:
class walkTheseWaysController:
    def __init__(self, robot, remote, checkpoint):
        self.remote = remote
        self.robot = robot
        self.cfg = loadParameters(checkpoint)
        self.policy = Policy(checkpoint)
        self.command_profile = CommandInterface()
        self.agent = WalkTheseWaysAgent(self.cfg, self.command_profile, self.robot)
        self.agent = HistoryWrapper(self.agent)
        self.hist_data = {}
        self.init()

    def init(self):
        self.obs = self.agent.reset()
        self.policy_info = {}
        self.command_profile.yaw_vel_cmd = 0.0
        self.command_profile.x_vel_cmd = 0.0
        self.command_profile.y_vel_cmd = 0.0
        self.command_profile.stance_width_cmd=0.25
        self.command_profile.footswing_height_cmd=0.08
        self.command_profile.step_frequency_cmd = 3.0
        self.command_profile.bodyHeight = 0.00

    def update(self, robot, remote):
        action = self.policy(self.obs, self.policy_info)
        self.obs, self.ret, self.done, self.info = self.agent.step(action)
        for key, value in self.info.items():
            if key in self.hist_data:
                self.hist_data[key].append(value)
            else:
                self.hist_data[key] = [value]

In [ ]:
checkpoint = "../Go2Py/assets/checkpoints/walk_these_ways/"
controller = walkTheseWaysController(robot, remote, checkpoint)
fsm = FSM(robot, remote, safety_hypervisor, user_controller_callback=controller.update)

## Set Initial Gait Parameters

In [7]:
controller.command_profile.pitch_cmd=0.0
controller.command_profile.body_height_cmd=0.0
controller.command_profile.footswing_height_cmd=0.08
controller.command_profile.roll_cmd=0.0
controller.command_profile.stance_width_cmd=0.2
controller.command_profile.x_vel_cmd=0.3  # Forward velocity
controller.command_profile.y_vel_cmd=0.0
controller.command_profile.setGaitType("trotting")

## Control Commands

Pressing `u` on the keyboard will make the robot stand up. This is equivalent to the `L2+A` combo of the Go2 builtin state machine. After the robot is on its feet, pressing `s` will hand over the control to the RL policy. This action is equivalent to the `start` key of the builtin controller. When you want to stop, pressing `u` again will act similarly to the real robot and locks it in standing mode. Finally, pressing `u` again will command the robot to sit down.

**Keyboard Controls:**
- `u` - Stand up / Stand down / Sit down (cycles through states)
- `s` - Start RL policy control

## Change Velocity Commands (Run these while robot is walking)

In [ ]:
# Move forward
controller.command_profile.x_vel_cmd=0.6
controller.command_profile.y_vel_cmd=0.00

In [ ]:
# Move backward
controller.command_profile.x_vel_cmd=-0.6
controller.command_profile.y_vel_cmd=0.00

In [ ]:
# Strafe right
controller.command_profile.x_vel_cmd=0.00
controller.command_profile.y_vel_cmd=0.6

In [ ]:
# Strafe left
controller.command_profile.x_vel_cmd=0.00
controller.command_profile.y_vel_cmd=-0.6

In [ ]:
# Rotate clockwise
controller.command_profile.yaw_vel_cmd = 1.0

In [ ]:
# Rotate counter-clockwise
controller.command_profile.yaw_vel_cmd = -1.0

In [ ]:
# Combined movement: forward + rotate
vx = 0.5
vy = 0.0
omega = 0.6
controller.command_profile.x_vel_cmd = vx
controller.command_profile.y_vel_cmd = vy
controller.command_profile.yaw_vel_cmd = omega

In [ ]:
# Stop all movement
controller.command_profile.x_vel_cmd = 0.0
controller.command_profile.y_vel_cmd = 0.0
controller.command_profile.yaw_vel_cmd = 0.0

## Adjust Gait Parameters

In [ ]:
# Increase step frequency for faster gait
controller.command_profile.step_frequency_cmd = 3.5

In [ ]:
# Increase foot swing height for stepping over obstacles
controller.command_profile.footswing_height_cmd=0.18

## Data Collection and Analysis

In [ ]:
import time
contacts = []
feet_vels = []

for i in range(2000):
    contact_state = robot.getFootContact()>15
    sites = ['FR_foot', 'FL_foot', 'RR_foot', 'RL_foot']
    feet_vel = [np.linalg.norm(robot.getFootVelInWorld(s)) for s in sites]
    contacts.append(contact_state)
    feet_vels.append(feet_vel)
    time.sleep(0.01)

feet_vels = np.stack(feet_vels)
contacts = np.stack(contacts)

In [ ]:
import matplotlib.pyplot as plt
start = 250
end = 500
plt.plot(contacts[start:end,0])
plt.plot(feet_vels[start:end,0])
plt.legend(['contact state', 'foot velocity'])
plt.grid(True)
plt.tight_layout()
plt.savefig('walk.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot foot contact forces
foot_contact_forces_mag = np.array(controller.hist_data["foot_contact_forces_mag"])

foot_nb = foot_contact_forces_mag.shape[1]
n_cols = 3
n_rows = int(np.ceil(foot_nb / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
axes = axes.flatten()

for i in range(foot_nb):
    axes[i].plot(foot_contact_forces_mag[:, i])
    axes[i].set_title(f'Foot {i+1} Contact Force Magnitude')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel('Force Magnitude')

for j in range(foot_nb, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.savefig("foot_contact_profile.png")
plt.show()

## Close Simulation

In [ ]:
fsm.close()
robot.close()